# Prevendo os sobreviventes do Titanic
# Modelo 4

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [41]:
treino = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Data-Science\Titanic\train.csv')
teste = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Data-Science\Titanic\test.csv')

treino.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [42]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Concatenando, transformando e separando treino e teste

In [43]:
#Adicionar coluna com a origem
treino['Origem'] = 'treino'
teste['Origem'] = 'teste'

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,treino
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,treino
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,treino
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,treino
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,treino


In [44]:
# Concatenando os DataFrames
df_concatenado = pd.concat([treino, teste], ignore_index=True)
df_concatenado

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,treino
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,treino
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,treino
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,treino
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,treino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,teste
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,teste
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,teste
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,teste


### Transformações

In [45]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_concatenado['Embarked'] = label_encoder.fit_transform(df_concatenado['Embarked'])
df_concatenado['Sex'] = label_encoder.fit_transform(df_concatenado['Sex'])

df_concatenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2,treino
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,treino
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,treino
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2,treino
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2,treino


In [46]:
print(df_concatenado.isna().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          0
Origem            0
dtype: int64


In [47]:
df_concatenado['Age'].fillna(np.mean(df_concatenado['Age']), inplace=True)
df_concatenado['Fare'].fillna(np.mean(df_concatenado['Fare']), inplace=True)
df_concatenado['Embarked'].fillna(-1, inplace=True)

### Separando os dataframes

In [48]:
df_concatenado.shape

(1309, 13)

In [49]:
# Separando os DataFrames com base na coluna de origem
treino = df_concatenado[df_concatenado['Origem'] == 'treino'].drop(columns=['Origem'])
teste = df_concatenado[df_concatenado['Origem'] == 'teste'].drop(columns=['Origem', 'Survived'])

treino.shape

(891, 12)

In [50]:
teste.shape

(418, 11)

In [51]:
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
891,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,1
892,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,2
893,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,1
894,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,2
895,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,2


In [52]:
print(teste.isna().sum())
print()
print(treino.isna().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


## Selecionando as features

In [53]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

In [54]:
from sklearn.model_selection import train_test_split
X_treino, X_valid, y_treino, y_valid = train_test_split(treino[features], treino[target], random_state=10, test_size=0.5)

## Verificando parâmetros para a DecisionTreeClassifier

In [63]:
def get_acc(Xtrain, Xvalid, ytrain, yvalid, SamplesLeaf=[10], LeafNodes=[4], RandomState=0):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import accuracy_score
    
    maior_acc_samples = 0
    melhor_sample = SamplesLeaf[0]
    for samples in SamplesLeaf:
        modelo = DecisionTreeClassifier(random_state=RandomState, min_samples_leaf=samples)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_samples:
            maior_acc_samples = accuracy
            melhor_sample = samples

    maior_acc_nodes = 0
    melhor_nodes = LeafNodes[0]

    for nodes in LeafNodes:
        modelo = DecisionTreeClassifier(random_state=RandomState, max_leaf_nodes=nodes)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_nodes:
            maior_acc_nodes = accuracy
            melhor_nodes = nodes

    print(f'Melhor min_samples_leaf: {melhor_sample} com {maior_acc_samples:.4f} de acurácia')
    print(f'Melhor max_leaf_nodes: {melhor_nodes} com {maior_acc_nodes:.4f} de acurácia')


In [64]:
get_acc(X_treino, X_valid, y_treino, y_valid, SamplesLeaf=list(range(2,20, 2)), LeafNodes=list(range(2, 400, 2)))

Melhor min_samples_leaf: 14 com 0.8251 de acurácia
Melhor max_leaf_nodes: 8 com 0.8318 de acurácia


## Treinando o modelo4 com max_leaf_nodes = 8

In [65]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo4 = DecisionTreeClassifier(random_state=0, max_leaf_nodes=8)
modelo4.fit(X_treino, y_treino)
y_pred4 = modelo4.predict(X_teste)

y_pred4

array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [69]:
#Estava em float
y_pred4 = y_pred4.astype(int)
y_pred4

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão

In [70]:
sub = pd.Series(y_pred4, index=teste['PassengerId'], name='Survived')
sub.shape

(418,)

In [71]:
sub.to_csv('Quarto_modelo.csv', header=True)